In [3]:
import ipywidgets as widgets
from IPython.display import display

# Decorador para registrar operaciones con su símbolo
def register_op(symbol):
    def decorator(func):
        func.op_symbol = symbol  # Agrega un atributo al método
        return func
    return decorator

class Calculator:
    def __init__(self):
        # Estado interno de la calculadora
        self.display = '0'
        self.first_operand = None
        self.operator = None
        self.waiting_for_second_operand = False

        # Widget de visualización (solo lectura)
        self.display_widget = widgets.Text(
            value=self.display,
            disabled=True,
            layout=widgets.Layout(width='100%', height='40px'),
            style={'font_size': '24px'}
        )

        # Definición de botones en una cuadrícula similar
        btn_layout = widgets.Layout(width='60px', height='40px')
        # Lista de botones a mostrar; se incluyen dígitos, punto, clear y operadores
        btn_texts = [
            ['C', '7', '8', '9'],
            ['/', '4', '5', '6'],
            ['*', '1', '2', '3'],
            ['-', '0', '.', '+'],
            ['=']
        ]

        # Lista para almacenar cada fila de botones (HBox)
        self.button_rows = []
        for row in btn_texts:
            btns = []
            for text in row:
                btn = widgets.Button(
                    description=text,
                    layout=btn_layout
                )
                # Asignamos la función de manejo de clics
                btn.on_click(self.on_button_click)
                btns.append(btn)
            self.button_rows.append(widgets.HBox(btns))

        # Componemos la interfaz vertical: display arriba y filas de botones abajo
        self.ui = widgets.VBox([self.display_widget] + self.button_rows)

        # Generamos dinámicamente el diccionario de operaciones usando reflexión
        self.operations = self.get_operations()

    def get_operations(self):
        """
        Busca en la instancia métodos que tengan el atributo 'op_symbol'
        y los registra en un diccionario: {symbol: método}.
        """
        ops = {}
        for attr_name in dir(self):
            attr = getattr(self, attr_name)
            if callable(attr) and hasattr(attr, 'op_symbol'):
                ops[getattr(attr, 'op_symbol')] = attr
        return ops

    def on_button_click(self, b):
        label = b.description
        if label == 'C':
            self.clear()
        elif label in '0123456789':
            self.input_digit(label)
        elif label == '.':
            self.input_decimal()
        elif label in ['+', '-', '*', '/', '=']:
            self.perform_operation(label)
        self.update_display()

    def input_digit(self, digit):
        if self.waiting_for_second_operand:
            self.display = digit
            self.waiting_for_second_operand = False
        else:
            # Si la pantalla muestra '0', se reemplaza
            self.display = digit if self.display == '0' else self.display + digit

    def input_decimal(self):
        if self.waiting_for_second_operand:
            self.display = '0.'
            self.waiting_for_second_operand = False
        elif '.' not in self.display:
            self.display += '.'

    def clear(self):
        self.display = '0'
        self.first_operand = None
        self.operator = None
        self.waiting_for_second_operand = False

    def perform_operation(self, next_operator):
        try:
            input_value = float(self.display)
        except ValueError:
            self.clear()
            return

        if self.first_operand is None:
            self.first_operand = input_value
        elif self.operator:
            result = self.calculate(self.first_operand, input_value, self.operator)
            self.display = str(result)
            self.first_operand = result

        # Si se pulsa '=' se reinicia el operador
        self.waiting_for_second_operand = True
        self.operator = None if next_operator == '=' else next_operator

    def calculate(self, first_operand, second_operand, operator):
        """
        Utiliza reflexión para llamar a la función de operación registrada.
        Si no se encuentra la operación, retorna el segundo operando.
        """
        if operator in self.operations:
            return self.operations[operator](first_operand, second_operand)
        else:
            return second_operand

    # Métodos de operación registrados mediante el decorador

    @register_op('+')
    def op_add(self, a, b):
        return a + b

    @register_op('-')
    def op_sub(self, a, b):
        return a - b

    @register_op('*')
    def op_mul(self, a, b):
        return a * b

    @register_op('/')
    def op_div(self, a, b):
        return a / b if b != 0 else "Error"

    def update_display(self):
        self.display_widget.value = self.display

    def show(self):
        display(self.ui)

# Instanciar y mostrar la calculadora en Google Colab
calc = Calculator()
calc.show()
